In [29]:
import wbdata
import pandas as pd
import dash
from dash import dcc, html, dash_table, Input, Output
import plotly.express as px
import pycountry
import numpy as np

# Define countries (using ISO2 codes for data fetching)
countries = [
    'AT', 'BE', 'BG', 'HR', 'CY', 'CZ', 'DK', 'EE', 'FI', 'FR', 'DE', 'GR', 'HU',
    'IE', 'IT', 'LV', 'LT', 'LU', 'MT', 'NL', 'PL', 'PT', 'RO', 'SK', 'SI', 'ES', 'SE',
    'CH', 'GB', 'NO', 'US', 'CA', 'AU', 'NZ', 'CN', 'JP'
]

# Define indicators for GDP and its components, including Population
indicators = {
    'NY.GDP.MKTP.CD': 'GDP_Total',
    'NE.CON.PRVT.CD': 'Private_Consumption_C',
    'NE.GDI.TOTL.CD': 'Investment_I',
    'NE.CON.GOVT.CD': 'Government_Spending_G',
    'NE.RSB.GNFS.CD': 'Net_Exports_NX',
    'SP.POP.TOTL': 'Population'
}

# Fetch data (latest available)
data = wbdata.get_dataframe(indicators, country=countries)
data.reset_index(inplace=True)
latest_data = data.sort_values('date').groupby('country').tail(1)
latest_data.dropna(subset=['GDP_Total'], inplace=True)

# Convert currency columns to Trillion dollars scale (instead of billion)
for col in ['GDP_Total', 'Private_Consumption_C', 'Investment_I', 'Government_Spending_G', 'Net_Exports_NX']:
    latest_data[col] /= 1e12

# Calculate percentages for the pillars relative to GDP_Total
for pillar in ['Private_Consumption_C', 'Investment_I', 'Government_Spending_G', 'Net_Exports_NX']:
    latest_data[pillar + '_Percent'] = latest_data[pillar] / latest_data['GDP_Total'] * 100
    latest_data[pillar + '_Formatted'] = latest_data.apply(
        lambda row: f"{row[pillar]:.2f} ({row[pillar + '_Percent']:.2f}%)", axis=1
    )

# Calculate Private Consumption Per Capita (in thousand dollars)
# Private Consumption (in trillion dollars) -> convert back to dollars: *1e12,
# then per capita = (private consumption dollars / population) and finally /1e3 for thousand dollars.
latest_data['Private_Consumption_Per_Capita'] = (latest_data['Private_Consumption_C'] * 1e12 / latest_data['Population']) / 1e3
latest_data['Private_Consumption_Per_Capita'] = latest_data['Private_Consumption_Per_Capita'].round(2)

# Helper function to convert country name to ISO3 code
def get_iso3(country_name):
    try:
        return pycountry.countries.search_fuzzy(country_name)[0].alpha_3
    except Exception:
        return None

# Create a new column with ISO3 codes for Dash choropleth
latest_data['country_code'] = latest_data['country'].apply(get_iso3)

# Define region mapping based on country names
north_america_europe = {
    "Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus", "Czech Republic",
    "Denmark", "Estonia", "Finland", "France", "Germany", "Greece", "Hungary",
    "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg", "Malta", "Netherlands",
    "Poland", "Portugal", "Romania", "Slovakia", "Slovenia", "Spain", "Sweden",
    "Switzerland", "United Kingdom", "Norway", "United States", "Canada"
}
asia_oceania = {"China", "Japan", "Australia", "New Zealand"}

# Add region column based on country name
latest_data['region'] = latest_data['country'].apply(
    lambda x: "North America & Europe" if x in north_america_europe 
    else ("Asia & Oceania" if x in asia_oceania else "Other")
)

# Dash app setup
app = dash.Dash(__name__)

# Layout with two dropdowns: one for the indicator selector and one for region filtering.
app.layout = html.Div([
    html.H2("GDP Components Interactive Table & Geographical Heatmap"),
    html.Div([
        html.Label("Select Indicator:"),
        dcc.Dropdown(
            id='column-selector',
            options=[
                {'label': 'GDP Total (Trillion $)', 'value': 'GDP_Total'},
                {'label': 'Private Consumption (%)', 'value': 'Private_Consumption_C_Percent'},
                {'label': 'Investment (%)', 'value': 'Investment_I_Percent'},
                {'label': 'Government Spending (%)', 'value': 'Government_Spending_G_Percent'},
                {'label': 'Net Exports (%)', 'value': 'Net_Exports_NX_Percent'}
            ],
            value='GDP_Total'
        )
    ], style={'width': '45%', 'display': 'inline-block'}),
    html.Div([
        html.Label("Select Region:"),
        dcc.Dropdown(
            id='region-selector',
            options=[
                {'label': 'All', 'value': 'All'},
                {'label': 'North America & Europe', 'value': 'North America & Europe'},
                {'label': 'Asia & Oceania', 'value': 'Asia & Oceania'}
            ],
            value='All'
        )
    ], style={'width': '45%', 'display': 'inline-block', 'float': 'right'}),
    dash_table.DataTable(
        id='data-table',
        columns=[
            {"name": "Country", "id": "country"},
            {"name": "Year", "id": "date"},
            {"name": "GDP Total (Trillion $)", "id": "GDP_Total"},
            {"name": "Private Consumption", "id": "Private_Consumption_C_Formatted"},
            {"name": "Private Consumption Per Capita (Thousand $)", "id": "Private_Consumption_Per_Capita"},
            {"name": "Investment", "id": "Investment_I_Formatted"},
            {"name": "Government Spending", "id": "Government_Spending_G_Formatted"},
            {"name": "Net Exports", "id": "Net_Exports_NX_Formatted"}
        ],
        data=latest_data.to_dict('records'),
        sort_action='custom',
        sort_mode='multi',
        style_table={'overflowX': 'auto'}
    ),
    dcc.Graph(id='heatmap')
])

# Callback to update both the table and the heatmap based on region, indicator, and sorting
@app.callback(
    [Output('data-table', 'data'),
     Output('heatmap', 'figure')],
    [Input('region-selector', 'value'),
     Input('column-selector', 'value'),
     Input('data-table', 'sort_by')]
)
def update_table_and_heatmap(region, selected_column, sort_by):
    # Filter based on region
    if region != "All":
        filtered_data = latest_data[latest_data['region'] == region].copy()
    else:
        filtered_data = latest_data.copy()
        
    # Apply custom sorting: only use the last sorting action (reset previous sorts)
    if sort_by:
        sort_item = sort_by[-1]
        col = sort_item["column_id"]
        direction = sort_item["direction"]
        if col.endswith('_Formatted'):
            sort_col = col.replace('_Formatted', '_Percent')
        else:
            sort_col = col
        filtered_data = filtered_data.sort_values(by=sort_col, ascending=(direction == 'asc'))
    
    # Define hover data for the choropleth (showing all five columns' percentage values)
    hover_info = {
        "GDP_Total": True,
        "Private_Consumption_C_Percent": True,
        "Investment_I_Percent": True,
        "Government_Spending_G_Percent": True,
        "Net_Exports_NX_Percent": True,
        "country_code": False
    }
    
    # Build the heat map figure (using raw GDP_Total in trillion $)
    if selected_column == 'GDP_Total':
        fig = px.choropleth(
            filtered_data,
            locations='country_code',
            color='GDP_Total',
            hover_name='country',
            hover_data=hover_info,
            color_continuous_scale='Blues',
            projection='natural earth',
            title='Geographical Heatmap: GDP Total (Trillion $)'
        )
    else:
        fig = px.choropleth(
            filtered_data,
            locations='country_code',
            color=selected_column,
            hover_name='country',
            hover_data=hover_info,
            color_continuous_scale='Blues',
            projection='natural earth',
            title=f'Geographical Heatmap: {selected_column.replace("_", " ")}'
        )
    return filtered_data.to_dict('records'), fig

if __name__ == '__main__':
    app.run_server(debug=True)
